In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from scipy.integrate import solve_ivp
from matplotlib import colors
import pickle
from scipy.signal import find_peaks
from scipy.stats import pearsonr
import HD_utils.circular_stats as cstat
from HD_utils.network import *
from HD_utils.matrix import *
from HD_utils.adap_sim_move import *
from HD_utils.adap_sim_stable import *
from HD_utils.IO import *
from HD_utils.plot import *
from HD_utils.comput_property import *

In [2]:
# Simulation theta precision
theta_num = 50
dtheta = (2*np.pi)/theta_num
theta_range = np.arange(-np.pi+dtheta/2, np.pi, dtheta)
# Default parameters
actfun = max0x
net_diff_equa = net_diff_equa_f_in
inputs = np.array([-1, -0.6, -0.3, -0.1, 0, 0.1, 0.3, 0.6, 1])
zeroid = np.where(inputs == 0)[0][0]
b0 = 1
tau = 20 # ms

# ONE RING

In [3]:
# Changeable parameters
ring_num = 1
weight_fun = asym_vw_wde_general
search_pars = {'JI': np.linspace(-100,0,26), 'JE': np.linspace(0,100,26)}
file_pre_name = '65_1'
# Generated parameters
par_num = len(search_pars)
par_names = list(search_pars.keys())
search_num = len(ParameterGrid(search_pars))

network_evals, network_evaldes, network_acvs, network_pars, network_ts = load_pickle(
    ['evals', 'eval_des', 'acvs', 'pars', 'ts'], weight_fun, actfun, file_pre_name)

Vels, network_eval_moving, network_eval_moving_des, network_acvs_moving, network_ts_moving, network_eval_moving_sum = load_pickle(
    ['moving_slope', 'moving_eval', 'moving_eval_des', 'moving_acvs', 'moving_ts', 'moving_eval_sum'], weight_fun, actfun, file_pre_name)

# Variables calculation
total_num = len(network_evals)
valid_index_s = np.where(network_evals == 'valid')[0]
valid_num = len(valid_index_s)
stable_mov_range, stable_mov_range_id, linear_mov_range, linear_mov_range_id = cal_linear_range(network_eval_moving, Vels, inputs, valid_index_s)
valid_index_part_linear = np.where( linear_mov_range[:,1] > 0 )[0]
valid_index_linear_move = np.where( linear_mov_range[:,1] == 1 )[0]
valid_index_stable_move = np.where( stable_mov_range[:,1] == 1 )[0]
linear_num = len(valid_index_linear_move)

print('N   VS  SM  LM')
print(total_num, len(valid_index_s), len(valid_index_stable_move), linear_num)

corr_stat_list, ratio_stat_list, slope = cal_print_increase_u_f(valid_index_linear_move, network_acvs_moving, inputs, zeroid, actfun, b0)

N   VS  SM  LM
676 366 366 366
Net num: 366
peak u Slope: +% = 317 (86.6%), +M = 0.059, +SD = 0.027 +MAX = 0.099, -M = -0.977, -SD = 1.377
    
mean u Slope: +% = 366 (100.0%), +M = 1.410, +SD = 3.632 +MAX = 31.217, -M = nan, -SD = nan
    
peak f Slope: +% = 317 (86.6%), +M = 0.059, +SD = 0.027 +MAX = 0.099, -M = -0.977, -SD = 1.377
peak f Ratio:                   +M = 0.336, +SD = 0.134 +MAX = 0.501, -M = -0.310, -SD = 0.224
mean f Slope: +% = 339 (92.6%), +M = 0.021, +SD = 0.010 +MAX = 0.083, -M = -0.168, -SD = 0.210
mean f Ratio:                   +M = 0.708, +SD = 0.245 +MAX = 1.011, -M = -0.320, -SD = 0.211
All min except mean u
       Slope: +% = 317 (86.6%), +M = 0.020, +SD = 0.008 +MAX = 0.057, -M = -0.977, -SD = 1.377
       Ratio:                   +M = 0.336, +SD = 0.134 +MAX = 0.501, -M = -0.310, -SD = 0.224
All min
       Slope: +% = 317 (86.6%), +M = 0.020, +SD = 0.008 +MAX = 0.057, -M = -0.977, -SD = 1.377
    


In [4]:
stat_acv_increase = pd.DataFrame(columns=['Class', 'Ring No.', 'N: valid stationary', 'N: linearly integrates', 'Statistics', 'No. +', 'M,+', "SD,+", "Max,+",  "M,-", "SD,-"])
aclass = 'Synapse-modulation'
aring_num = 1
N_valid = len(valid_index_s)
N_linear = linear_num

stat_names = ['Peak u slope', 'Mean u slope', 'Peak f slope', 'Peak f ratio', 'Mean f slope', 'Mean f ratio', 'Min of the three (excluding mean u) slope', 'Min of the four slope']
stat_list = [[int(corr_stat_list[0][0])] + list(corr_stat_list[0][2:]), [int(corr_stat_list[1][0])] + list(corr_stat_list[1][2:]), [int(corr_stat_list[2][0])] + list(corr_stat_list[2][2:]), [int(corr_stat_list[2][0])] + list(ratio_stat_list[2]), [int(corr_stat_list[3][0])] + list(corr_stat_list[3][2:]), [int(corr_stat_list[3][0])] + list(ratio_stat_list[3]), [int(corr_stat_list[4][0])] + list(corr_stat_list[4][2:]), [int(corr_stat_list[5][0])] + list(corr_stat_list[5][2:])]

for i, stat in enumerate(stat_names):
    stat_acv_increase.loc[len(stat_acv_increase)] = [aclass, aring_num, N_valid, N_linear, stat] + list(stat_list[i])

In [5]:
# Symmetry of the central ring
index_shape_mismatch, dev_ratios, if_match = cal_central_shape_match_loop(network_acvs_moving, valid_index_s, zeroid, ring_num)
cmatch_pro, mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev = 100-len(index_shape_mismatch)/len(valid_index_s) * 100, \
    np.mean(dev_ratios), np.std(dev_ratios), np.max(dev_ratios)
print(f'Symmetry of the central ring: {cmatch_pro:.1f}%   {mean_cmatch_dev:.3e} ± {sd_cmatch_dev:.3e} MAX:{max_cmatch_dev:.3e}')

Symmetry of the central ring: 100.0%   2.072e-04 ± 9.452e-04 MAX:2.234e-02


# TWO RING

In [6]:
# Changeable parameters
ring_num = 2
weight_fun = cos_weight3
search_pars = {'JI': np.linspace(-100,0,11), 'JE': np.linspace(0,100,11), 'K0': np.linspace(-100,80,10)}
file_pre_name = '65_2_3'
# Default parameters
phi = np.pi * 1/9
# Generated parameters
par_num = len(search_pars)
search_num = len(ParameterGrid(search_pars))
par_names = list(search_pars.keys())

# This data comes from a deleted notebook, need update later
network_evals, network_evaldes, network_acvs, network_pars, network_ts = load_pickle(
    ['evals', 'eval_des', 'acvs', 'pars', 'ts'], weight_fun, actfun, '65_2')
valid_index_s = np.where(network_evals == 'valid')[0]

Vels, network_eval_moving, network_eval_moving_des, network_acvs_moving, network_ts_moving, network_eval_moving_sum = load_pickle(
    ['moving_slope', 'moving_eval', 'moving_eval_des', 'moving_acvs', 'moving_ts', 'moving_eval_sum'], weight_fun, actfun, file_pre_name)

# Variables calculation
total_num = len(network_evals)
valid_index_s = np.where(network_evals == 'valid')[0]
valid_num = len(valid_index_s)
stable_mov_range, stable_mov_range_id, linear_mov_range, linear_mov_range_id = cal_linear_range(network_eval_moving, Vels, inputs, valid_index_s)
valid_index_part_linear = np.where( linear_mov_range[:,1] > 0 )[0]
valid_index_linear_move = np.where( linear_mov_range[:,1] == 1 )[0]
valid_index_stable_move = np.where( stable_mov_range[:,1] == 1 )[0]
linear_num = len(valid_index_linear_move)

print('N    VS  PL  SM  LM')
print(total_num, len(valid_index_s), len(valid_index_part_linear), len(valid_index_stable_move), linear_num)

corr_stat_list, ratio_stat_list, slope  = cal_print_increase_u_f_2(valid_index_linear_move, network_acvs_moving, inputs, zeroid, actfun, b0)

N    VS  PL  SM  LM
1210 253 243 129 129
Net num: 129
peak u Slope: +% = 126 (97.7%), +M = 0.240, +SD = 0.338 +MAX = 2.682, -M = -0.016, -SD = 0.016
    
mean u Slope: +% =   0 (0.0%), +M = nan, +SD = nan +MAX = nan, -M = -1.837, -SD = 3.314
    
peak f Slope: +% = 129 (100.0%), +M = 0.235, +SD = 0.335 +MAX = 2.682, -M = nan, -SD = nan
peak f Ratio:                   +M = 0.445, +SD = 0.082 +MAX = 0.539, -M = nan, -SD = nan
mean f Slope: +% = 129 (100.0%), +M = 0.027, +SD = 0.032 +MAX = 0.237, -M = nan, -SD = nan
mean f Ratio:                   +M = 0.502, +SD = 0.012 +MAX = 0.579, -M = nan, -SD = nan
All min except mean u
       Slope: +% = 126 (97.7%), +M = 0.028, +SD = 0.033 +MAX = 0.237, -M = -0.016, -SD = 0.016
       Ratio:                   +M = 0.448, +SD = 0.077 +MAX = 0.507, -M = -0.136, -SD = 0.149
All min
       Slope: +% =   0 (0.0%), +M = nan, +SD = nan +MAX = nan, -M = -1.837, -SD = 3.314
    


In [7]:
aclass = 'Shifter-ring'
aring_num = 2
N_valid = len(valid_index_s)
N_linear = linear_num

stat_names = ['Peak u slope', 'Mean u slope', 'Peak f slope', 'Peak f ratio', 'Mean f slope', 'Mean f ratio', 'Min of the three (excluding mean u) slope', 'Min of the four slope']
stat_list = [[int(corr_stat_list[0][0])] + list(corr_stat_list[0][2:]), [int(corr_stat_list[1][0])] + list(corr_stat_list[1][2:]), [int(corr_stat_list[2][0])] + list(corr_stat_list[2][2:]), [int(corr_stat_list[2][0])] + list(ratio_stat_list[2]), [int(corr_stat_list[3][0])] + list(corr_stat_list[3][2:]), [int(corr_stat_list[3][0])] + list(ratio_stat_list[3]), [int(corr_stat_list[4][0])] + list(corr_stat_list[4][2:]), [int(corr_stat_list[5][0])] + list(corr_stat_list[5][2:])]

for i, stat in enumerate(stat_names):
    stat_acv_increase.loc[len(stat_acv_increase)] = [aclass, aring_num, N_valid, N_linear, stat] + list(stat_list[i])

In [8]:
stat_2ring = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)', 'Mean: RL Sym $d_{rel}$', 'SD: RL Sym $d_{rel}$', 'Max: RL Sym $d_{rel}$'])

structure = "Shifter-ring (two rings, speed modulation)"
wfun_eq = False

# Input - R-L correlation
bump_amplitudes = cal_firate_a_acv_mean_a_peak(network_acvs_moving, inputs, valid_index_part_linear, b0, actfun, kind='increaseb0')
input_diff_cors, input_diff_ps = cal_input_diff_cor(inputs, bump_amplitudes[4:], valid_index_part_linear, linear_mov_range_id) 
show_value = input_diff_cors[valid_index_part_linear]
cor_list = []
for i in range(4):
    cor_list.append(np.nanmean(show_value[:,i]))
    cor_list.append(np.nanstd(show_value[:,i]))
    cor_list.append(np.nanmin(show_value[:,i]))
    
index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_match_loop(network_acvs_moving, valid_index_stable_move, zeroid)
mean_match_dev, sd_match_dev, max_match_dev = np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)

stat_2ring.loc[0] = [structure, ring_num, wfun_eq, waf_df_names[weight_fun.__name__], waf_df_names[actfun.__name__], total_num, valid_num, linear_num] + cor_list + [mean_match_dev, sd_match_dev, max_match_dev]

In [9]:
pd.set_option('display.max_rows', 22)
pd.set_option('display.float_format', '{:.3e}'.format)
stat_2ring.iloc[:,5:]

,N,N: valid stationary,N: linearly integrates,Mean: RL Corr (A),SD: RL Corr (A),Min RL Corr (A),Mean: RL Corr (B),SD: RL Corr (B),Min RL Corr (B),Mean: RL Corr (C),SD: RL Corr (C),Min RL Corr (C),Mean: RL Corr (D),SD: RL Corr (D),Min RL Corr (D),Mean: RL Sym $d_{rel}$,SD: RL Sym $d_{rel}$,Max: RL Sym $d_{rel}$
0,1210,253,129,9.994e-01,2.559e-03,9.773e-01,9.994e-01,2.559e-03,9.773e-01,9.999e-01,2.021e-04,9.988e-01,9.999e-01,4.485e-04,9.956e-01,5.984e-06,6.482e-05,4.108e-03


# THREE RING


In [10]:
# Changeable parameters
ring_num = 3
weight_fun = cos_weight_3r_Icos_s
search_pars = {'CI': np.linspace(2,20,10), 'CE': np.linspace(2,20,10), 'LI': np.linspace(0,5,11)}
file_pre_name = '65_3'
# Default parameters
alpha = -1/9*np.pi
bE = 5
bI = 0
# Generated parameters
par_num = len(search_pars)
search_num = len(ParameterGrid(search_pars))
par_names = list(search_pars.keys())

network_evals, network_evaldes, network_acvs, network_pars, network_ts = load_pickle(
    ['evals', 'eval_des', 'acvs', 'pars', 'ts'], weight_fun, actfun, "63_3_1_copy")
valid_index_s = np.where(network_evals == 'valid')[0]

Vels, network_eval_moving, network_eval_moving_des, network_acvs_moving, network_ts_moving, network_eval_moving_sum = load_pickle(
    ['moving_slope', 'moving_eval', 'moving_eval_des', 'moving_acvs', 'moving_ts', 'moving_eval_sum'], weight_fun, actfun, file_pre_name)

# Variables calculation
total_num = len(network_evals)
valid_index_s = np.where(network_evals == 'valid')[0]
valid_num = len(valid_index_s)
stable_mov_range, stable_mov_range_id, linear_mov_range, linear_mov_range_id = cal_linear_range(network_eval_moving, Vels, inputs, valid_index_s)
valid_index_part_linear = np.where( linear_mov_range[:,1] > 0 )[0]
valid_index_linear_move = np.where( linear_mov_range[:,1] == 1 )[0]
valid_index_stable_move = np.where( stable_mov_range[:,1] == 1 )[0]
linear_num = len(valid_index_linear_move)

print('N    VS   PL   SM   LM')
print(total_num, len(valid_index_s), len(valid_index_part_linear), len(valid_index_stable_move), linear_num)

corr_stat_list, ratio_stat_list, slope = cal_print_increase_u_f_3(valid_index_linear_move, network_acvs_moving, inputs, zeroid, actfun, [bE, bI])

N    VS   PL   SM   LM
1100 1096 1083 139 139
Net num: 139
Central peak u Slope: +% = 133 (95.7%), +M = 1.800, +SD = 0.735 +MAX = 2.869, -M = -0.543, -SD = 0.136
    
rotational peak u Slope: +% = 133 (95.7%), +M = 5.748, +SD = 3.017 +MAX = 12.197, -M = -1.267, -SD = 0.298
    
Central mean u Slope: +% =   7 (5.0%), +M = 4.774, +SD = 1.971 +MAX = 6.168, -M = -4.550, -SD = 1.862
    
rotational mean u Slope: +% =  54 (38.8%), +M = 0.686, +SD = 0.452 +MAX = 1.838, -M = -2.484, -SD = 2.163
    
Central peak f Slope: +% = 133 (95.7%), +M = 1.800, +SD = 0.735 +MAX = 2.869, -M = -0.543, -SD = 0.136
Central peak f Ratio:                   +M = 0.807, +SD = 0.148 +MAX = 0.949, -M = -0.603, -SD = 0.107
rotational peak f Slope: +% = 133 (95.7%), +M = 5.748, +SD = 3.017 +MAX = 12.197, -M = -0.938, -SD = 0.226
rotational peak f Ratio:                   +M = 0.777, +SD = 0.148 +MAX = 0.937, -M = -0.483, -SD = 0.098
Central mean f Slope: +% = 133 (95.7%), +M = 0.312, +SD = 0.173 +MAX = 0.754, -M = -

In [11]:
aclass = 'Shifter-ring'
aring_num = 3
N_valid = len(valid_index_s)
N_linear = linear_num

stat_names1 = ['Peak symmetric u slope', 'Peak shifter u slope', 'Mean symmetric u slope', 'Mean shifter u slope', 'Peak symmetric f slope', 'Peak shifter f slope', 'Mean symmetric f slope', 'Mean shifter f slope', 'Min of the three (excluding mean u) slope', 'Min of the four slope']
stat1_list = [[int(corr_stat_list[i][0])] + list(corr_stat_list[i][2:]) for i in range(len(corr_stat_list))]

stat_names2 = ['Peak symmetric f ratio', 'Peak shifter f ratio', 'Mean symmetric f ratio', 'Mean shifter f ratio']
stat2_list = [[int(corr_stat_list[i][0])] + list(ratio_stat_list[i]) for i in [4,5,6,7]]

for i, stat in enumerate(stat_names1):
    stat_acv_increase.loc[len(stat_acv_increase)] = [aclass, aring_num, N_valid, N_linear, stat] + list(stat1_list[i])
    
for i, stat in enumerate(stat_names2):
    stat_acv_increase.loc[len(stat_acv_increase)] = [aclass, aring_num, N_valid, N_linear, stat] + list(stat2_list[i])
    
stat_acv_increase.to_csv(TABLE_PATH /'Supplementary table 5. Speed modulation on Activity.csv')

In [12]:
stat_3ring = pd.DataFrame(columns=['Class', 'Ring No.', 'Weight Func equality', 'Weight Func', 'Act Func', 'N', 'N: valid stationary', 'N: linearly integrates', 'Mean: RL Corr (A)', 'SD: RL Corr (A)', 'Min RL Corr (A)', 'Mean: RL Corr (B)', 'SD: RL Corr (B)', 'Min RL Corr (B)', 'Mean: RL Corr (C)', 'SD: RL Corr (C)', 'Min RL Corr (C)', 'Mean: RL Corr (D)', 'SD: RL Corr (D)', 'Min RL Corr (D)',  'Mean: RL Sym $d_{rel}$', 'SD: RL Sym $d_{rel}$', 'Max: RL Sym $d_{rel}$', 'Mean: Sym ring Sym $d_{rel}$', 'SD: Sym ring Sym $d_{rel}$', 'Max: Sym ring Sym $d_{rel}$'])

structure = 'Shifter-ring (three rings, speed modulation)'

# Input - R-L correlation
bump_amplitudes = cal_firate_a_acv_mean_a_peak(network_acvs_moving, inputs, valid_index_part_linear, [bE, bI], actfun, kind='increaseb0')
input_diff_cors, input_diff_ps = cal_input_diff_cor(inputs, bump_amplitudes[4:], valid_index_part_linear, linear_mov_range_id) 
show_value = input_diff_cors[valid_index_part_linear]
cor_list = []
for i in range(4):
    cor_list.append(np.nanmean(show_value[:,i]))
    cor_list.append(np.nanstd(show_value[:,i]))
    cor_list.append(np.nanmin(show_value[:,i]))
    
# Mirror symmetry of the left and right rings
index_shape_mismatch, dev_shape_ratios, if_match = cal_lr_shape_match_loop(network_acvs_moving, valid_index_stable_move, zeroid)
lr_match_pro, mean_lrmatch_dev, sd_lrmatch_dev, max_lrmatch_dev = 100-len(index_shape_mismatch)/len(valid_index_stable_move) * 100, \
    np.mean(dev_shape_ratios), np.std(dev_shape_ratios), np.max(dev_shape_ratios)
    
# Mirror symmetry of the central ring
index_shape_mismatch, dev_ratios, if_match = cal_central_shape_match_loop(network_acvs_moving, valid_index_stable_move, zeroid)
mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev = np.mean(dev_ratios), np.std(dev_ratios), np.max(dev_ratios)

stat_3ring.loc[0] = [structure, ring_num, wfun_eq, waf_df_names[weight_fun.__name__], waf_df_names[actfun.__name__], total_num, valid_num, linear_num] + cor_list + [mean_lrmatch_dev, sd_lrmatch_dev, max_lrmatch_dev, mean_cmatch_dev, sd_cmatch_dev, max_cmatch_dev]

In [13]:
stat_3ring.iloc[:,8:]

,Mean: RL Corr (A),SD: RL Corr (A),Min RL Corr (A),Mean: RL Corr (B),SD: RL Corr (B),Min RL Corr (B),Mean: RL Corr (C),SD: RL Corr (C),Min RL Corr (C),Mean: RL Corr (D),SD: RL Corr (D),Min RL Corr (D),Mean: RL Sym $d_{rel}$,SD: RL Sym $d_{rel}$,Max: RL Sym $d_{rel}$,Mean: Sym ring Sym $d_{rel}$,SD: Sym ring Sym $d_{rel}$,Max: Sym ring Sym $d_{rel}$
0,9.998e-01,7.610e-04,9.897e-01,9.990e-01,6.121e-03,8.862e-01,9.955e-01,2.430e-02,6.802e-01,9.977e-01,1.151e-02,8.300e-01,1.180e-06,2.021e-05,1.174e-03,5.447e-07,1.295e-05,8.942e-04


In [14]:
# Input - Vel slope
num_neg_slope, num_pos_slope, meann, stdn, maxvn, minvn, meanp, stdp, maxvp, minvp = cal_vel_slop_stat(Vels, valid_index_linear_move, inputs)